In [1]:
from google.colab import files
uploaded = files.upload()

Saving login.csv to login.csv
Saving application_usage.csv to application_usage.csv
Saving file_acces.csv to file_acces.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report


In [3]:
ls

application_usage.csv  file_acces.csv  login.csv  sample_data/


In [17]:
import pandas as pd

login = pd.read_csv("login.csv")

# Display shape
print("Shape:", login.shape)

# Display first 5 rows as a table (Jupyter/IPython will auto-render as a nice table)
from IPython.display import display
display(login.head(5))


Shape: (20000, 7)


,user,department,role,device_id,geo_location,login,logout
0,user_0165,Finance,Engineer,device_168,UK,18-09-2025 08:07,18-09-2025 16:07
1,user_0083,IT,Intern,device_122,UK,10-09-2025 08:05,10-09-2025 15:05
2,user_0269,HR,Admin,device_088,US,18-09-2025 09:59,18-09-2025 18:59
3,user_0021,IT,Engineer,device_135,SG,27-09-2025 10:15,27-09-2025 19:15
4,user_0068,IT,Analyst,device_152,SG,22-09-2025 17:21,23-09-2025 13:21


In [18]:
import pandas as pd

file_access = pd.read_csv("file_acces.csv")

# Display shape
print("Shape:", file_access.shape)

# Display first 5 rows as a table
from IPython.display import display
display(file_access.head(5))


Shape: (20000, 7)


,user,department,role,file,category,action,access_time
0,user_0216,Sales,Manager,normal_0448.txt,normal,read,01-09-2025 23:10
1,user_0171,IT,Admin,normal_0382.txt,normal,read,22-09-2025 23:52
2,user_0105,R&D,Admin,auth_0008.docx,authenticated,read,10-09-2025 08:11
3,user_0071,R&D,Admin,auth_0221.docx,authenticated,delete,05-09-2025 15:52
4,user_0025,IT,Intern,private_0011.docx,private,read,29-09-2025 23:28


In [19]:
import pandas as pd

application_usage = pd.read_csv("application_usage.csv")

# Display shape
print("Shape:", application_usage.shape)

# Display first 5 rows as a table
from IPython.display import display
display(application_usage.head(5))


Shape: (20000, 13)


,user,department,role,device_id,device_type,application,start_time,end_time,duration_sec,network_in_kb,network_out_kb,cpu_usage,memory_usage
0,user_0014,Sales,Analyst,device_152,desktop,CloudDrive,20-09-2025 05:09,20-09-2025 05:41,1892,1045,828,72.48,61.05
1,user_0112,HR,Intern,device_111,mobile,IDE,21-09-2025 15:04,21-09-2025 15:58,3256,1307,1309,34.86,79.35
2,user_0072,Finance,Intern,device_069,desktop,FileTransferTool,19-09-2025 16:30,19-09-2025 17:15,2686,627,349,47.26,13.89
3,user_0174,IT,Manager,device_092,mobile,Browser,30-09-2025 12:02,30-09-2025 12:13,697,182,102,43.24,58.47
4,user_0260,Sales,Admin,device_195,laptop,AdminConsole,30-09-2025 03:29,30-09-2025 04:17,2847,659,835,59.65,94.68


In [22]:
from IPython.display import display

# Convert datetime columns for login.csv
login['login']  = pd.to_datetime(login['login'],  errors='coerce')
login['logout'] = pd.to_datetime(login['logout'], errors='coerce')

print("Null counts - login.csv")
display(login.isnull().sum().to_frame(name="Null_Count"))
display(login.head(5))

Null counts - login.csv


,Null_Count
user,0
department,0
role,0
device_id,0
geo_location,0
login,0
logout,0


,user,department,role,device_id,geo_location,login,logout
0,user_0165,Finance,Engineer,device_168,UK,2025-09-18 08:07:00,2025-09-18 16:07:00
1,user_0083,IT,Intern,device_122,UK,2025-09-10 08:05:00,2025-09-10 15:05:00
2,user_0269,HR,Admin,device_088,US,2025-09-18 09:59:00,2025-09-18 18:59:00
3,user_0021,IT,Engineer,device_135,SG,2025-09-27 10:15:00,2025-09-27 19:15:00
4,user_0068,IT,Analyst,device_152,SG,2025-09-22 17:21:00,2025-09-23 13:21:00


In [23]:
# Convert datetime columns for file_access.csv
file_access['access_time'] = pd.to_datetime(file_access['access_time'], errors='coerce')

print("Null counts - file_access.csv")
display(file_access.isnull().sum().to_frame(name="Null_Count"))
display(file_access.head(5))

Null counts - file_access.csv


,Null_Count
user,0
department,0
role,0
file,0
category,0
action,0
access_time,12089


,user,department,role,file,category,action,access_time
0,user_0216,Sales,Manager,normal_0448.txt,normal,read,2025-01-09 23:10:00
1,user_0171,IT,Admin,normal_0382.txt,normal,read,NaT
2,user_0105,R&D,Admin,auth_0008.docx,authenticated,read,2025-10-09 08:11:00
3,user_0071,R&D,Admin,auth_0221.docx,authenticated,delete,2025-05-09 15:52:00
4,user_0025,IT,Intern,private_0011.docx,private,read,NaT


In [24]:
# Convert datetime columns for application_usage.csv
application_usage['start_time'] = pd.to_datetime(application_usage['start_time'], errors='coerce')
application_usage['end_time']   = pd.to_datetime(application_usage['end_time'],   errors='coerce')

print("Null counts - application_usage.csv")
display(application_usage.isnull().sum().to_frame(name="Null_Count"))
display(application_usage.head(5))


Null counts - application_usage.csv


,Null_Count
user,0
department,0
role,0
device_id,0
device_type,0
application,0
start_time,0
end_time,0
duration_sec,0
network_in_kb,0


,user,department,role,device_id,device_type,application,start_time,end_time,duration_sec,network_in_kb,network_out_kb,cpu_usage,memory_usage
0,user_0014,Sales,Analyst,device_152,desktop,CloudDrive,2025-09-20 05:09:00,2025-09-20 05:41:00,1892,1045,828,72.48,61.05
1,user_0112,HR,Intern,device_111,mobile,IDE,2025-09-21 15:04:00,2025-09-21 15:58:00,3256,1307,1309,34.86,79.35
2,user_0072,Finance,Intern,device_069,desktop,FileTransferTool,2025-09-19 16:30:00,2025-09-19 17:15:00,2686,627,349,47.26,13.89
3,user_0174,IT,Manager,device_092,mobile,Browser,2025-09-30 12:02:00,2025-09-30 12:13:00,697,182,102,43.24,58.47
4,user_0260,Sales,Admin,device_195,laptop,AdminConsole,2025-09-30 03:29:00,2025-09-30 04:17:00,2847,659,835,59.65,94.68


## **Logins**

In [30]:
from IPython.display import display

# 1️⃣ Daily login counts per user
login['date'] = login['login'].dt.date
feat_logins = (
    login.groupby(['user', 'date'])
         .agg({'user': 'count'})
         .rename(columns={'user': 'login_count'})
         .reset_index()
)


In [31]:
# 2️⃣ Mean login and logout hours
login['login_hour']  = login['login'].dt.hour
login['logout_hour'] = login['logout'].dt.hour
feat_login_hours = (
    login.groupby('user')
         .agg({'login_hour':'mean', 'logout_hour':'mean'})
         .rename(columns={'login_hour':'mean_login_hour', 'logout_hour':'mean_logout_hour'})
         .reset_index()
)

In [32]:
# 3️⃣ Session duration
login['session_duration_min'] = (login['logout'] - login['login']).dt.total_seconds() / 60
feat_session_duration = (
    login.groupby('user')
         .agg(avg_session_min=('session_duration_min','mean'),
              total_session_min=('session_duration_min','sum'))
         .reset_index()
)

In [33]:
# 4️⃣ Unique devices & geo-locations
feat_device_geo = (
    login.groupby('user')
         .agg(unique_devices=('device_id','nunique'),
              unique_geo_locations=('geo_location','nunique'))
         .reset_index()
)


In [34]:
# 5️⃣ Day-of-week & weekend activity
login['login_dayofweek'] = login['login'].dt.dayofweek
feat_dayofweek = (
    login.groupby('user')
         .agg(avg_login_day=('login_dayofweek','mean'),
              weekend_logins=('login_dayofweek', lambda x: (x>=5).sum()))
         .reset_index()
)

In [35]:
# Merge all features into one dataset
features_login = (
    feat_logins
    .merge(feat_login_hours, on='user', how='left')
    .merge(feat_session_duration, on='user', how='left')
    .merge(feat_device_geo, on='user', how='left')
    .merge(feat_dayofweek, on='user', how='left')
)

display(features_login.head(5))

,user,date,login_count,mean_login_hour,mean_logout_hour,avg_session_min,total_session_min,unique_devices,unique_geo_locations,avg_login_day,weekend_logins
0,user_0001,2025-09-01,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,20
1,user_0001,2025-09-02,1,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,20
2,user_0001,2025-09-03,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,20
3,user_0001,2025-09-04,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,20
4,user_0001,2025-09-05,1,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,20


## **file_access**

In [36]:
from IPython.display import display

# 1️⃣ Count of file accesses per user
feat_file_counts = (
    file_access.groupby('user')
               .size()
               .reset_index(name='total_file_accesses')
)

In [37]:
# 2️⃣ Count of accesses by category per user
feat_file_category = (
    file_access.groupby(['user', 'category'])
               .size()
               .unstack(fill_value=0)
               .reset_index()
)

In [38]:
# Optional: flatten columns
feat_file_category.columns = ['user'] + [f'category_{col}' for col in feat_file_category.columns[1:]]

In [40]:
# 3️⃣ Count of actions (read/write/etc.) per user
feat_file_action = (
    file_access.groupby(['user', 'action'])
               .size()
               .unstack(fill_value=0)
               .reset_index()
)
feat_file_action.columns = ['user'] + [f'action_{col}' for col in feat_file_action.columns[1:]]

In [42]:
# 4️⃣ Daily file access counts
file_access['access_date'] = file_access['access_time'].dt.date
feat_file_daily = (
    file_access.groupby(['user', 'access_date'])
               .size()
               .reset_index(name='daily_file_access_count')
)
# Average daily access per user
feat_file_daily_avg = (
    feat_file_daily.groupby('user')
                   .agg(avg_daily_file_access=('daily_file_access_count','mean'))
                   .reset_index()
)

In [43]:
# 5️⃣ Merge all features
features_file_access = (
    feat_file_counts
    .merge(feat_file_category, on='user', how='left')
    .merge(feat_file_action, on='user', how='left')
    .merge(feat_file_daily_avg, on='user', how='left')
)

display(features_file_access.head(5))

,user,total_file_accesses,category_authenticated,category_normal,category_private,action_delete,action_read,action_write,avg_daily_file_access
0,user_0001,77,29,38,10,24,31,22,2.416667
1,user_0002,73,27,35,11,28,26,19,2.600000
2,user_0003,80,24,43,13,22,30,28,2.666667
3,user_0004,83,23,48,12,24,23,36,3.333333
4,user_0005,85,26,47,12,27,34,24,3.166667


## **application usage**

In [44]:
from IPython.display import display

# 1️⃣ Total and average duration of application usage per user (in minutes)
application_usage['duration_min'] = application_usage['duration_sec'] / 60
feat_app_duration = (
    application_usage.groupby('user')
                     .agg(total_usage_min=('duration_min','sum'),
                          avg_usage_min=('duration_min','mean'))
                     .reset_index()
)

In [45]:
# 2️⃣ Count of applications used per user
feat_app_count = (
    application_usage.groupby('user')
                     .agg(unique_apps=('application','nunique'))
                     .reset_index()
)

In [47]:
# 3️⃣ Count by device type per user
feat_device_type = (
    application_usage.groupby(['user','device_type'])
                     .size()
                     .unstack(fill_value=0)
                     .reset_index()
)
feat_device_type.columns = ['user'] + [f'device_{col}' for col in feat_device_type.columns[1:]]

In [48]:
# 4️⃣ Average resource usage per user
feat_resource_usage = (
    application_usage.groupby('user')
                     .agg(avg_cpu=('cpu_usage','mean'),
                          avg_memory=('memory_usage','mean'),
                          avg_network_in_kb=('network_in_kb','mean'),
                          avg_network_out_kb=('network_out_kb','mean'))
                     .reset_index()
)

In [49]:
# 5️⃣ Daily application usage count per user
application_usage['usage_date'] = application_usage['start_time'].dt.date
feat_daily_usage = (
    application_usage.groupby(['user','usage_date'])
                     .size()
                     .reset_index(name='daily_app_usage_count')
)
# Average daily usage
feat_daily_usage_avg = (
    feat_daily_usage.groupby('user')
                    .agg(avg_daily_app_usage=('daily_app_usage_count','mean'))
                    .reset_index()
)

In [50]:
# 6️⃣ Merge all features
features_application_usage = (
    feat_app_duration
    .merge(feat_app_count, on='user', how='left')
    .merge(feat_device_type, on='user', how='left')
    .merge(feat_resource_usage, on='user', how='left')
    .merge(feat_daily_usage_avg, on='user', how='left')
)

display(features_application_usage.head(5))

,user,total_usage_min,avg_usage_min,unique_apps,device_desktop,device_laptop,device_mobile,avg_cpu,avg_memory,avg_network_in_kb,avg_network_out_kb,avg_daily_app_usage
0,user_0001,2024.650000,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.260870,2.464286
1,user_0002,3133.916667,48.967448,8,17,27,20,43.442344,51.934531,1016.687500,1010.750000,2.206897
2,user_0003,2060.183333,33.228763,8,22,17,23,50.825645,46.598710,977.370968,1072.161290,2.296296
3,user_0004,2485.000000,33.133333,8,29,23,23,50.471333,47.140533,1066.880000,1124.266667,2.777778
4,user_0005,2019.400000,30.596970,8,30,18,18,45.814545,43.624848,913.363636,1022.151515,2.538462


# **Merge**

In [51]:
from functools import reduce
from IPython.display import display

# List of all feature DataFrames
dfs = [features_login, features_file_access, features_application_usage]

# Merge all on 'user'
merged_features = reduce(lambda left, right: pd.merge(left, right, on='user', how='outer'), dfs)

# Fill missing values if needed (optional)
merged_features.fillna(0, inplace=True)

# Display the merged dataset
display(merged_features.head(5))
print("Shape of merged dataset:", merged_features.shape)


,user,date,login_count,mean_login_hour,mean_logout_hour,avg_session_min,total_session_min,unique_devices,unique_geo_locations,avg_login_day,...,avg_usage_min,unique_apps,device_desktop,device_laptop,device_mobile,avg_cpu,avg_memory,avg_network_in_kb,avg_network_out_kb,avg_daily_app_usage
0,user_0001,2025-09-01,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,...,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.26087,2.464286
1,user_0001,2025-09-02,1,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,...,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.26087,2.464286
2,user_0001,2025-09-03,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,...,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.26087,2.464286
3,user_0001,2025-09-04,2,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,...,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.26087,2.464286
4,user_0001,2025-09-05,1,10.561404,18.140351,454.736842,25920.0,51,6,3.052632,...,29.342754,8,13,23,33,46.946377,44.953913,930.318841,1004.26087,2.464286


Shape of merged dataset: (8027, 30)


In [53]:
from google.colab import files  # only if using Colab
files.download("merged_user_features.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--------------------------------Over--------------------------------

In [ ]:
merged_features = features_logins.merge(feat_file_access, on=['user', 'date'], how='left')
merged_features = merged_features.merge(feat_out_of_session, on=['user', 'date'], how='left')
merged_features = merged_features.merge(feat_usb_usage, on=['user', 'date'], how='left')

# Fill NaN values with 0, as NaN indicates no activity for that user on that date
merged_features.fillna(0, inplace=True)

print("Merged features:")
print(merged_features.head())
print("\nMerged features shape:", merged_features.shape)

Merged features:
    user        date  login_count  mean_login_hour  mean_logout_hour  \
0  user1  2023-01-01            1         8.033333         15.933333   
1  user1  2023-01-02            1         8.033333         15.933333   
2  user1  2023-01-03            1         8.033333         15.933333   
3  user1  2023-01-04            1         8.033333         15.933333   
4  user1  2023-01-05            1         8.033333         15.933333   

   files_per_day  out_of_session_file_access  usb_connections_per_day  
0            1.0                         1.0                      0.0  
1            2.0                         2.0                      0.0  
2            3.0                         1.0                      0.0  
3            5.0                         4.0                      1.0  
4            2.0                         1.0                      1.0  

Merged features shape: (600, 8)


In [ ]:
merged_features.to_csv('merged_features.csv', index=False)
from google.colab import files
files.download('merged_features.csv')